This Notebook is based on TensorFlow Quantum:
A Software Framework for Quantum Machine Learning available from Tensforflow Quantum website.

## Setup:

In [ ]:
!pip install tensorflow-quantum

In [ ]:
# Update package resources to account for version changes.
import importlib, pkg_resources
importlib.reload(pkg_resources)

<module 'pkg_resources' from '/usr/local/lib/python3.7/dist-packages/pkg_resources/__init__.py'>

## Import TensorFlow and the module dependencies:

In [ ]:
import tensorflow as tf
import tensorflow_quantum as tfq

import cirq , random , sympy
import numpy as np

# visualization tools
%matplotlib inline
import matplotlib.pyplot as plt
from cirq.contrib.svg import SVGCircuit


The first step is to generate the quantum data. We can
use Cirq for this task. The common imports required for
working with TFQ are shown below:

The function below generates the quantum dataset; labels use a one-hot encoding:

In [ ]:

def generate_dataset (qubit , theta_a , theta_b , num_samples):
  q_data = []
  labels = []
  blob_size = abs( theta_a - theta_b ) / 5
  for _ in range ( num_samples ):
    coin = random.random ()
    spread_x , spread_y = np.random.uniform(-blob_size , blob_size , 2)
  if coin < 0.5:
    label = [1 , 0]
    angle = theta_a + spread_y
  else :
    label = [0 , 1]
    angle = theta_b + spread_y
  labels.append( label )
  q_data.append( cirq.Circuit (
    cirq.Ry (- angle )( qubit ) ,
    cirq.Rx (- spread_x ) ( qubit )))
  return( tfq.convert_to_tensor ( q_data ) ,
          np.array( labels ))

Generating a dataset and the associated labels after
picking some parameter values:

In [ ]:
# Generate the dataset:

qubit = cirq.GridQubit(0 , 0)
theta_a = 1
theta_b = 4
num_samples = 200
q_data , labels = generate_dataset(qubit , theta_a , theta_b , num_samples )

In [ ]:

#from sympy import theta
theta = sympy.Symbol('theta')
q_model = cirq.Circuit( cirq.Ry(theta) (qubit))
q_data_input = tf.keras.Input (shape=() , dtype = tf.dtypes.string )
expectation = tfq.layers.PQC (q_model , cirq.Z( qubit ))
expectation_output = expectation ( q_data_input )

The purpose of the rotation gate is to minimize the superposition from the input quantum data such that we
can get maximum useful information from the measurement. This quantum model is then attached to a small
classifier NN to complete our hybrid model. Notice in
the code below that quantum layers can appear among
classical layers inside a standard Keras model:

In [ ]:

classifier = tf . keras . layers . Dense (
    2, activation = tf . keras . activations . softmax )
classifier_output = classifier (
    expectation_output )
model = tf . keras . Model ( inputs = q_data_input ,
    outputs = classifier_output )

Next we can train this hybrid model on the quantum data
defined earlier. Below we use as our loss function the
cross entropy between the labels and the predictions of
the classical NN; the ADAM optimizer is chosen for parameter updates.

In [ ]:
# Train the Hybrid model:

optimizer = tf.keras.optimizers.Adam(learning_rate =0.1)
loss = tf.keras.losses.CategoricalCrossentropy()
model . compile ( optimizer = optimizer , loss = loss )
history = model . fit ( x= q_data , y= labels , epochs =50)

Finally, we can use our trained hybrid model to classify new quantum datapoints:

In [ ]:
# Classify new quantum datapoints:
test_data , _ = generate_dataset (
qubit , theta_a , theta_b , 1)
p = model . predict ( test_data ) [0]
print (f" prob (a)={p [0]:.4 f}, prob (b)={p [1]:.4 f}")